In [234]:
ls

MapNamesToLogs.ipynb  bball-ref.js          finalGameLog.py       gameLogs.ipynb        mapNames.py           players/
README.md*            cleanGameLogs.py      finalNames.py         includes/             node_modules/


In [235]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', 500)

logs = 'players/2015/logs/alllogs.csv'

games = pd.read_csv(logs)

In [236]:
games = games.rename(columns={'Unnamed: 5': 'Home'})
games = games.rename(columns={'Unnamed: 7': 'ScoreDiff'})
games = games.drop(['Rk', 'G'], axis=1)

In [237]:
games.Home = games.Home.fillna(value=1)
games.Home = games.Home.replace(to_replace='@', value=0)

In [238]:
games['ScoreDiff'] = games.apply(lambda row: (int(str(row['ScoreDiff']).split('(')[1].split(')')[0])), axis=1)

In [239]:
#games.dtypes

In [240]:
games['fptsFT'] = games.apply(lambda row: (row['FT']), axis=1)
games['fpts2P'] = games.apply(lambda row: (row['FG']*2), axis=1)
games['fpts3P'] = games.apply(lambda row: (row['3P']*3), axis=1)
games['fptsRebounds'] = games.apply(lambda row: (row['TRB']*1.2), axis=1)
games['fptsAssists'] = games.apply(lambda row: (row['AST']*1.5), axis=1)
games['fptsBlocks'] = games.apply(lambda row: (row['BLK']*2), axis=1)
games['fptsSteals'] = games.apply(lambda row: (row['STL']*2), axis=1)
games['fptsTurnovers'] = games.apply(lambda row: (row['TOV']*-1), axis=1)

games['fpts'] = games['fptsFT'] + games['fpts2P'] + games['fptsRebounds'] + games['fptsAssists'] + \
games['fptsBlocks'] + games['fptsSteals'] + games['fptsTurnovers']

In [251]:
n = 'players/2015/names/allnames.csv'
names = pd.read_csv(n)

In [252]:
#names

In [253]:
games = pd.merge(games, names, on='pname', how='outer')

In [254]:
games

,Date,Age,Tm,Home,Opp,ScoreDiff,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,pname,fptsFT,fpts2P,fpts3P,fptsRebounds,fptsAssists,fptsBlocks,fptsSteals,fptsTurnovers,fpts,Player_x,From_x,To_x,Pos_x,Ht_x,Wt_x,Birth Date_x,College_x,Player_y,From_y,To_y,Pos_y,Ht_y,Wt_y,Birth Date_y,College_y
0,2014-10-29,24-023,NYK,1,CHI,-24,0,21:28,3,6,0.500,0,0,NaN,0,0,NaN,2,4,6,1,0,1,2,4,6,3.4,-15,acyqu01,0,6,0,7.2,1.5,2,0,-2,14.7,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
1,2014-10-30,24-024,NYK,0,CLE,5,1,22:27,4,7,0.571,0,0,NaN,0,0,NaN,6,4,10,2,0,1,2,6,8,7.8,2,acyqu01,0,8,0,12.0,3.0,2,0,-2,23.0,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
2,2014-11-02,24-027,NYK,1,CHO,3,1,20:20,1,4,0.250,0,1,0.000,0,0,NaN,0,3,3,2,0,0,0,2,2,1.1,11,acyqu01,0,2,0,3.6,3.0,0,0,0,8.6,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
3,2014-11-05,24-030,NYK,0,DET,-3,0,13:42,0,1,0.000,0,0,NaN,0,0,NaN,2,4,6,1,0,1,2,1,0,0.9,-6,acyqu01,0,0,0,7.2,1.5,2,0,-2,8.7,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
4,2014-11-07,24-032,NYK,0,BRK,-11,0,22:14,3,7,0.429,0,0,NaN,1,2,0.500,6,3,9,0,0,0,3,3,7,3.8,-15,acyqu01,1,6,0,10.8,0.0,0,0,-3,14.8,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
5,2014-11-08,24-033,NYK,0,ATL,-7,0,13:34,1,1,1.000,0,0,NaN,0,0,NaN,1,2,3,0,0,0,1,2,2,1.2,-10,acyqu01,0,2,0,3.6,0.0,0,0,-1,4.6,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
6,2014-11-12,24-037,NYK,1,ORL,-2,1,20:12,1,1,1.000,0,0,NaN,0,0,NaN,2,2,4,1,0,0,2,4,2,0.8,-8,acyqu01,0,2,0,4.8,1.5,0,0,-2,6.3,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
7,2014-11-14,24-039,NYK,1,UTA,-2,0,19:39,2,2,1.000,0,0,NaN,2,2,1.000,1,1,2,0,0,0,0,3,6,5.2,-5,acyqu01,2,4,0,2.4,0.0,0,0,0,8.4,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
8,2014-11-16,24-041,NYK,1,DEN,16,1,31:10,3,7,0.429,0,1,0.000,2,2,1.000,1,7,8,3,3,0,0,3,8,11.0,-3,acyqu01,2,6,0,9.6,4.5,0,6,0,28.1,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University
9,2014-11-19,24-044,NYK,0,MIN,-16,1,32:18,4,8,0.500,0,0,NaN,1,1,1.000,2,6,8,0,0,0,3,2,9,4.4,-4,acyqu01,1,8,0,9.6,0.0,0,0,-3,15.6,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University,Quincy Acy,2013,2016,F,6-7,240,October 6 1990,Baylor University


In [255]:
games.describe()

,Home,ScoreDiff,GS,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,fptsFT,fpts2P,fpts3P,fptsRebounds,fptsAssists,fptsBlocks,fptsSteals,fptsTurnovers,fpts,From_x,To_x,Wt_x,From_y,To_y,Wt_y
count,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,12885.000000,13466.000000,13466.000000,8927.000000,13466.000000,13466.000000,7969.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13464.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13466.000000,13290.000000,13290.000000,13290.000000,13290.000000,13290.000000,13290.000000
mean,0.504233,0.196718,0.498069,3.702659,8.318357,0.433538,0.830833,2.367592,0.321615,1.783677,2.337517,0.753486,1.001411,3.102777,4.104188,2.198277,0.782861,0.466360,1.384673,1.958191,10.019828,7.568484,0.156046,1.783677,7.405317,2.492500,4.925026,3.297416,0.932719,1.565721,-1.384673,18.525204,2009.511738,2015.835666,217.913093,2009.511738,2015.835666,217.913093
std,0.500001,14.102036,0.500015,2.979213,5.640131,0.230599,1.213449,2.542362,0.298557,2.322573,2.844856,0.280508,1.449765,2.750765,3.638153,2.391540,1.021822,0.891652,1.435619,1.458969,7.890429,7.094651,10.779592,2.322573,5.958426,3.640346,4.365783,3.587311,1.783304,2.043644,1.435619,12.716436,4.159517,0.370592,28.345626,4.159517,0.370592,28.345626
min,0.000000,-53.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-8.000000,-40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,-3.000000,1996.000000,2015.000000,161.000000,1996.000000,2015.000000,161.000000
25%,0.000000,-10.000000,0.000000,1.000000,4.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,2.000000,-7.000000,0.000000,2.000000,0.000000,1.200000,0.000000,0.000000,0.000000,-2.000000,8.600000,2007.000000,2016.000000,195.000000,2007.000000,2016.000000,195.000000
50%,1.000000,1.000000,0.000000,3.000000,7.000000,0.429000,0.000000,2.000000,0.333000,1.000000,2.000000,0.833000,0.000000,2.000000,3.000000,1.000000,0.000000,0.000000,1.000000,2.000000,9.000000,6.200000,0.000000,1.000000,6.000000,0.000000,3.600000,1.500000,0.000000,0.000000,-1.000000,16.700000,2010.000000,2016.000000,220.000000,2010.000000,2016.000000,220.000000
75%,1.000000,10.000000,1.000000,6.000000,12.000000,0.563000,1.000000,4.000000,0.500000,3.000000,4.000000,1.000000,1.000000,4.000000,6.000000,3.000000,1.000000,1.000000,2.000000,3.000000,15.000000,11.875000,7.000000,3.000000,12.000000,3.000000,7.200000,4.500000,2.000000,2.000000,0.000000,26.400000,2013.000000,2016.000000,240.000000,2013.000000,2016.000000,240.000000
max,1.000000,53.000000,1.000000,17.000000,37.000000,1.000000,10.000000,16.000000,1.000000,18.000000,20.000000,1.000000,15.000000,20.000000,25.000000,17.000000,8.000000,9.000000,10.000000,6.000000,51.000000,41.900000,45.000000,18.000000,34.000000,30.000000,30.000000,25.500000,18.000000,16.000000,0.000000,81.300000,2016.000000,2016.000000,360.000000,2016.000000,2016.000000,360.000000


In [250]:
#games.dtypes